## How to Run Games with Goad

### 0. Imports

In [1]:
import sys
if '../../' not in sys.path:
    sys.path.append('../../')

from src.go_defence.kata_gtp_subprocess import Kata_GTP_Subprocess
from src.go_defence.subprocess_game import Subprocess_Game
from src.go_defence.kata_adv_gtp_subprocess import Kata_Adv_GTP_Subprocess
from src.go_defence.gnu_go_subprocess import GNU_Go_Subprocess
from src.go_defence.kata_and_gnu_subprocess import Kata_and_GNU_Subprocess
from src.go_defence.kata_player import Kata_Player
from src.go_defence.defended_model import Defended_Model
from src.go_defence.gnu_go_subprocess import GNU_Go_Subprocess
from src.adv_detection.adv_det_model import Model as adv_det_model
from subprocess import Popen, PIPE, STDOUT

/Users/yz24943/Documents/Research/Summer_Project/Go_Adversarial_Policy_Defence/venv/lib/python3.11/site-packages/torch/cuda/__init__.py:63: FutureWarning: The pynvml package is deprecated. Please install nvidia-ml-py instead. If you did not install pynvml directly, please report this to the maintainers of the package that installed pynvml for you.
  import pynvml  # type: ignore[import]


### 0.1 Defining File Locations

If you are getting broken pipe errors incorrect file locations are the most likely cause.

In [ ]:
big_adv = '../../models/adversarial/big-adversary-s216m/model.bin.gz'
cont_adv = '../../models/adversarial/continuous-adversary-s630m/model.bin.gz'

model = "../../models/kataGo/bin_files/kata1-b18c384nbt-s9996604416-d4316597426.bin.gz"
b10_chkpt = '../../models/kataGo/control-b10-s419m/model.ckpt'
may_24_path = '../../models/kataGo/ckpt_files/kata1-b18c384nbt-s9996604416-d4316597426/model.ckpt'
adv_det_model = '../../models/detection/model_0.chkpt'

adv_config = '../../configs/sabaki/gtp-adv600-vm1-s.cfg'
config = "../../configs/katago/victim.cfg"

### 1.0 Instance Subprocesses

All of the models are capable of outputting in the form of GTP (Go Text Protocol). We run each model as it's own isolated subprocess then use Subprocess_Game as an interface for the models to play against each other using GTP.

In [ ]:
gnu = GNU_Go_Subprocess()
Kata = Kata_GTP_Subprocess('may_24', model, config)

#### 1.1 Run the Game (takes ~5 min)

Pass the subprocesses to Subprocess_Game, model1 will be black (pla) and model2 will be white (opp). Then call game() to run the a game. print_mode will determine if the moves and boardstates of the subprocesses are printed. The outputs are useful for debugging and demonstration.

In [ ]:
game = Subprocess_Game(model1 = gnu, model2 = Kata)

result,sgf = game.game(print_mode=True)

print(result)

print(sgf)

### 2.0 Running with GoAD and Adversarial Policy

Both these models have quirks which mean they're instanced differently.

#### 2.1 Instancing an Adversarial Policy

The adversarial policies use AMCTS (Adversarial Monte Carlo Tree Search). This means they use a version of the victim model when performing tree searches. So when instancing the adversarial policy we need to pass it a copy of it's victim model.

In [3]:
big = Kata_Adv_GTP_Subprocess('big_adv', big_adv, adv_config, model)

#### 2.2 instancing GoAD

GoAD is an ensemble using three models. A detection model, the victim model and a rules based model (GnuGO).
However in implementation have to instance the victim model twice in two different ways needing two different files:
1. As a subprocess (.bin.gz)
2. As a pytorch model (.chkpt)

This lets us have it play moves as a subprocess, and extracted it's activation to use in the detection model.

In [4]:
name = 'Goad'
detection_threshold = [8,10] #how many of the previous classification needed to be 1 to detect an adv pol. here it's 8/10
detection_window = [50,200] #The window of moves where detection is allowed

GoAD = Defended_Model(name, model, may_24_path, adv_det_model, detection_threshold, detection_window, config)

In [5]:
game = Subprocess_Game(model1 = big, model2 = GoAD)

result,sgf = game.game(print_mode=True)

print(result)

print(sgf)

FileNotFoundError: [Errno 2] No such file or directory: 'engines/KataGo-custom/cpp/katago'